# Prepare soil carbon data

In [ ]:
dir01 = '../paper_deficit/output/01_prep/'
dir02 = '../paper_deficit/output/02_dbase/'

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt

---

In [ ]:
# Load datasets
ds_soilgrids = xr.open_zarr(dir01 + 'ds_prep_soilgrids2020_ocs_0-30cm.zarr')
ds_gsoc = xr.open_zarr(dir01 + 'ds_prep_gsoc_ocs_0-30cm.zarr')

In [ ]:
# Merge data to one array
da_soc = xr.merge([ds_soilgrids['soilgrids2020_ocs_0-30cm'], 
                   ds_gsoc['gsoc_ocs_0-30cm']]) \
    .to_array('soc')

# Create SOC dataset
# Calculate arrays with min, mean, and max values for each grid cell
ds_soc = xr.Dataset({
    'soc_min': da_soc.min('soc'),
    'soc_mean': da_soc.mean('soc'),
    'soc_max': da_soc.max('soc')
})

# Export SOC dataset
ds_soc.to_zarr(dir02 + 'ds_prep_soc.zarr', mode='w');

---

### Check

In [ ]:
ds_soc = xr.open_zarr(dir02 + 'ds_prep_soc.zarr')
ds_area = xr.open_zarr(dir01 + 'ds_prep_area_ha.zarr')

In [ ]:
for i in list(ds_soc.data_vars):
    fig, ax = plt.subplots(figsize=(10, 5), ncols=1, nrows=1)
    ds_soc[i].plot.imshow(ax=ax, robust=True)
    ax.set_title(i)

In [ ]:
def get_pgc(da):
    v_pgc = ((da * ds_area.area_ha).sum(['lat', 'lon']) * 0.000000001) \
        .compute().item() 
    return round(v_pgc, 1)

In [ ]:
print('Soilgrids 2020: ' + str(get_pgc(ds_soilgrids['soilgrids2020_ocs_0-30cm'])))
print('GSOC: ' + str(get_pgc(ds_gsoc['gsoc_ocs_0-30cm'])))

In [ ]:
print('Max Scenario')
print('SOC max: ' + str(get_pgc(ds_soc.soc_max)))
print('SOC mean: ' + str(get_pgc(ds_soc.soc_mean)))
print('SOC min: ' + str(get_pgc(ds_soc.soc_min)))

In [ ]:
ds_soilgrids['soilgrids2020_ocs_0-30cm'].plot.imshow(robust=True)

In [ ]:
ds_gsoc['gsoc_ocs_0-30cm'].plot.imshow(robust=True)

---

In [ ]:
dir_old = '../paper_carbon_densities/output/03_fill_nans/'

In [ ]:
da_sgrids_old = xr.open_zarr(dir_old + 'ds_fillnan_sgrids_ocs_2020_0-30cm.zarr') \
    ['sgrids_ocs_2020_0-30cm']

da_sgrids_old.plot.imshow(robust=True)
print(get_pgc(da_sgrids_old))

In [ ]:
da_gsoc_old = xr.open_zarr(dir_old + 'ds_fillnan_gsoc_ocs_0-30cm.zarr') \
    ['gsoc_ocs_0-30cm']

da_gsoc_old.plot.imshow(robust=True)
print(get_pgc(da_gsoc_old))

In [ ]:
da_gsoc_reg = xr.open_zarr(dir01 + 'ds_regridded_gsoc_ocs_0-30cm.zarr') \
    ['regridded_gsoc_ocs_0-30cm']

da_gsoc_reg.plot.imshow(robust=True)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
(ds_gsoc['gsoc_ocs_0-30cm'] - da_gsoc_old) \
    .plot.imshow(ax=ax, robust=True)